In [1]:
import gym
import tensorflow as tf
import numpy as np
import copy
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
env.render()


SFFF
FHFH
FFFH
HFFG


In [2]:
max_episodes=100
max_step=100
lr=0.02
lr2=1.0
gamma=0.99
epsilon0 = 0.1

In [3]:
inputdim=env.observation_space.n
outputdim=env.action_space.n
hiddendim=100
print(inputdim, outputdim)
X=tf.placeholder(tf.float32, shape = [None,inputdim],name="X")
Y=tf.placeholder(tf.float32, shape = [None,outputdim],name="Y")

#weight1=tf.get_variable("w1", initializer=tf.contrib.layers.xavier_initializer(), shape=[inputdim,hiddendim], dtype=tf.float32)
#bias1=tf.get_variable("b1", initializer=tf.contrib.layers.xavier_initializer(), shape=[hiddendim], dtype=tf.float32)
#weight2=tf.get_variable("w2", initializer=tf.contrib.layers.xavier_initializer(), shape=[hiddendim,hiddendim], dtype=tf.float32)
#bias2=tf.get_variable("b2", initializer=tf.contrib.layers.xavier_initializer(), shape=[hiddendim], dtype=tf.float32)
#weight3=tf.get_variable("w3", initializer=tf.contrib.layers.xavier_initializer(), shape=[hiddendim,hiddendim], dtype=tf.float32)
#bias3=tf.get_variable("b3", initializer=tf.contrib.layers.xavier_initializer(), shape=[hiddendim], dtype=tf.float32)
#weight4=tf.get_variable("w4", initializer=tf.contrib.layers.xavier_initializer(), shape=[hiddendim,outputdim], dtype=tf.float32)
#bias4=tf.get_variable("b4", initializer=tf.contrib.layers.xavier_initializer(), shape=[outputdim], dtype=tf.float32)

#weight1=tf.get_variable("w1", initializer=tf.initializers.random_normal(0,0.05), shape=[inputdim,hiddendim], dtype=tf.float32)
#bias1=tf.get_variable("b1", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
#weight2=tf.get_variable("w2", initializer=tf.initializers.random_normal(0,0.05), shape=[hiddendim,hiddendim], dtype=tf.float32)
#bias2=tf.get_variable("b2", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
#weight3=tf.get_variable("w3", initializer=tf.initializers.random_normal(0,0.05), shape=[hiddendim,hiddendim], dtype=tf.float32)
#bias3=tf.get_variable("b3", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
#weight4=tf.get_variable("w4", initializer=tf.initializers.random_normal(0,0.05), shape=[hiddendim,outputdim], dtype=tf.float32)
#bias4=tf.get_variable("b4", initializer=tf.initializers.random_normal(0,0.00), shape=[outputdim], dtype=tf.float32)

weight1=tf.get_variable("w1", initializer=tf.keras.initializers.he_uniform(), shape=[inputdim,hiddendim], dtype=tf.float32)
bias1=tf.get_variable("b1", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight2=tf.get_variable("w2", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias2=tf.get_variable("b2", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight3=tf.get_variable("w3", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias3=tf.get_variable("b3", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight4=tf.get_variable("w4", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,outputdim], dtype=tf.float32)
bias4=tf.get_variable("b4", initializer=tf.initializers.random_normal(0,0.00), shape=[outputdim], dtype=tf.float32)

#varlist=[weight1,weight2,weight3,weight4,bias1,bias2,bias3,bias4]
varlist=[weight1,weight4,bias1,bias4]
H1=tf.nn.relu(tf.nn.xw_plus_b(X,weight1,bias1))
#H2=tf.nn.relu(tf.nn.xw_plus_b(H1,weight2,bias2))
#H3=tf.nn.relu(tf.nn.xw_plus_b(H2,weight3,bias3))
#Y_pred=tf.nn.xw_plus_b(H3,weight4,bias4)
Y_pred=tf.nn.xw_plus_b(H1,weight4,bias4)


loss=tf.reduce_sum(tf.square(Y-Y_pred))
opt=tf.train.AdamOptimizer(lr).minimize(loss,var_list=varlist)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


16 4


In [4]:
max_episodes=100
max_step=100
sess.run(init)
# initialize

epsilon=epsilon0
min_step=max_step
min_step_episode=max_episodes
min_step_path=[]
for episode in range(max_episodes):
    epsilon=epsilon*0.99
    state = env.reset()
    total_reward = 0
    state_list = [state]
    win=0

    for step in range(0,max_step):
        state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m[0][state]=1.0
        Q_pred=sess.run(Y_pred,feed_dict={X:state_m})
        if np.random.rand()>epsilon :
            action=np.argmax(Q_pred[0])
        else:
            action=np.random.choice(env.action_space.n)
        
        state_next,reward,game_over,_ = env.step(action)
        
        if reward==0 : 
            reward=-0.01
        if reward!=1.0 and game_over==1:
            reward=-1.0
            
        state_m_next=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m_next[0][state_next]=1.0
        Q_pred_next=sess.run(Y_pred, feed_dict={X:state_m_next}) 

        if np.random.rand()>epsilon :
            action_next=np.argmax(Q_pred_next[0])
        else:
            action_next=np.random.choice(env.action_space.n)

        Y_true=copy.deepcopy(Q_pred[0])

        if reward==1.0 and game_over==1:
            Y_true[action]= reward
        else :
            Y_true[action]=reward+gamma* (Q_pred_next[0][action_next])
        Y_true=np.reshape(Y_true,[1,env.action_space.n])
        for j in range(0,1):
            sess.run([opt],feed_dict={X:state_m,Y:Y_true})
            cost= sess.run([loss],feed_dict={X:state_m,Y:Y_true})
#            print(cost)
            
        
        
#        print(state, Q_pred,Y_true,action)
#        state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
#        state_m[0][state]=1.0
#        Q_pred=sess.run(Y_pred,feed_dict={X:state_m})
#        cost= sess.run([loss],feed_dict={X:state_m,Y:Y_true})
#        print(Q_pred,cost)
        state=state_next

        total_reward=total_reward+reward
        state_list.append(state)
        if game_over and reward==1:
            win=1
            if step< min_step:
                min_step=step
                min_step_path=state_list
                min_step_episode=episode
        if game_over==1:
            break


    line_out="Episode: %d, step: %d, total_reward: %f, epsilon: %f" %(episode, step, total_reward,epsilon)
    print(line_out)
    if reward==1 :
        print(state_list)
print( "min step: ", min_step, "min step episode", min_step_episode)
print("shortest path: ", min_step_path)


Episode: 0, step: 21, total_reward: -1.210000, epsilon: 0.099000
Episode: 1, step: 12, total_reward: -1.120000, epsilon: 0.098010
Episode: 2, step: 14, total_reward: -1.140000, epsilon: 0.097030
Episode: 3, step: 48, total_reward: -1.480000, epsilon: 0.096060
Episode: 4, step: 1, total_reward: -1.010000, epsilon: 0.095099
Episode: 5, step: 82, total_reward: -1.820000, epsilon: 0.094148
Episode: 6, step: 9, total_reward: -1.090000, epsilon: 0.093207
Episode: 7, step: 14, total_reward: 0.860000, epsilon: 0.092274
[0, 4, 8, 9, 8, 9, 8, 9, 13, 13, 13, 13, 14, 13, 14, 15]
Episode: 8, step: 1, total_reward: -1.010000, epsilon: 0.091352
Episode: 9, step: 19, total_reward: -1.190000, epsilon: 0.090438
Episode: 10, step: 3, total_reward: -1.030000, epsilon: 0.089534
Episode: 11, step: 17, total_reward: -1.170000, epsilon: 0.088638
Episode: 12, step: 1, total_reward: -1.010000, epsilon: 0.087752
Episode: 13, step: 6, total_reward: -1.060000, epsilon: 0.086875
Episode: 14, step: 5, total_reward: 

Episode: 92, step: 7, total_reward: 0.930000, epsilon: 0.039271
[0, 4, 8, 9, 13, 14, 10, 14, 15]
Episode: 93, step: 5, total_reward: 0.950000, epsilon: 0.038878
[0, 4, 8, 9, 13, 14, 15]
Episode: 94, step: 5, total_reward: 0.950000, epsilon: 0.038490
[0, 4, 8, 9, 13, 14, 15]
Episode: 95, step: 5, total_reward: 0.950000, epsilon: 0.038105
[0, 4, 8, 9, 13, 14, 15]
Episode: 96, step: 7, total_reward: 0.930000, epsilon: 0.037724
[0, 4, 8, 8, 8, 9, 13, 14, 15]
Episode: 97, step: 5, total_reward: 0.950000, epsilon: 0.037346
[0, 4, 8, 9, 13, 14, 15]
Episode: 98, step: 5, total_reward: 0.950000, epsilon: 0.036973
[0, 4, 8, 9, 13, 14, 15]
Episode: 99, step: 5, total_reward: 0.950000, epsilon: 0.036603
[0, 4, 8, 9, 13, 14, 15]
min step:  5 min step episode 14
shortest path:  [0, 4, 8, 9, 13, 14, 15]


In [5]:
#state=np.arange(0,15)
#sess.run(init)
state_m=np.zeros([16,env.observation_space.n],dtype=np.float32)
for i in range(0,15):
    state_m[i][i]=1.0
print(state_m)
Q_pred=sess.run(Y_pred,feed_dict={X:state_m})
print(Q_pred)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 0.04214111  0.7884078  -0.19296977 -0.20802286]
 [ 0.02253047 -1.7311689  -0.10026854 -1.2524462 ]
 [-0.00903405  0.77869    -0.84523964 -0.02939156]
 [-0.5327989  -0.92588234 -1.3